# DS-SF-34 | 11 | Logistic Regression | Assignment | Starter Code

In [42]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

from sklearn import linear_model, cross_validation

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [6]:
def odds(p):
    return p/(1.-p)

def log_odds(p):
    odds_value = odds(p)
    return np.log(odds_value)

print "probability:", 0.32
print "odds:", odds(0.32)
print "log odds:", log_odds(0.32)

probability: 0.32
odds: 0.470588235294
log odds: -0.753771802376


In [7]:
def sigmoid(log_odds):
    return 1. / (1 +np.exp(-log_odds))

sigmoid(log_odds(0.32))
sigmoid(-0.7527)

0.32023326917809214

## Probability, Odds, and Odds Ratios

**Probability:** The number of ways that an event can occur divided by the total number of possible outcomes.

The probability of drawing a red card from a standard deck of cards is 26/52 (50 percent).
The probability of drawing a club from that deck is 13/52 (25 percent).

> ### Question 1.  What's the probability of getting heads in a fair coin flip?

In [ ]:
# TODO

The odds for an event is the ratio of the number of ways the event can occur compared to the number of ways it does not occur.

For example, using the same events as above, the odds for:
- Drawing a red card from a standard deck of cards is 1:1; and
- Drawing a club from that deck is 1:3.

> ### Question 2.  What's the odds of a fair coin flip?

In [ ]:
# TODO

> ### Question 3.  Suppose that 18 out of 20 patients in an experiment lost weight while using diet A, while 16 out of 20  lost weight using diet B.  What's the probability of weight loss with diet A?  What's the odds?

In [ ]:
# TODO

> ### Question 4.  What's the probablity of weight loss with diet B?  What are the odds?

In [ ]:
# TODO

> ### Question 5.  What's the odds ratio?

In [ ]:
# TODO

## Bank Marketing

In [10]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-11-bank-marketing.csv'))

In [11]:
df

,age,job,marital,education,default,...,campaign,pdays,previous,poutcome,c
0,30,unemployed,married,primary,no,...,1,-1,0,unknown,no
1,33,services,married,secondary,no,...,1,339,4,failure,no
2,35,management,single,tertiary,no,...,1,330,1,failure,no
3,30,management,married,tertiary,no,...,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,...,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,...,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,...,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,...,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,...,4,211,3,other,no


> The dataset is related to the direct marketing campaigns (by phone) of a Portuguese banking institution.  The classification goal is to predict if the client will subscribe a term deposit (variable y).

Attributes Information:

- Input variables:
  - [Bank client data]
    - `age` (numeric)
    - `job`: type of job (categorical)
    - `marital`: marital status (categorical)
      - Note: `divorced` means divorced or widowed)
    - `education` (categorical)
    - `default`: has credit in default? (categorical)
    - `balance`: bank account balance (\$)
    - `housing`: has housing loan? (categorical)
    - `loan`: has personal loan? (categorical)
  - [Data related with the last contact of the current campaign]
    - `contact`: contact communication type (categorical)
    - `month`: last contact month of year (categorical)
    - `day_of_week`: last contact day of the week (categorical)
    - `duration`: last contact duration, in seconds (numeric)
      - Important note: this attribute highly affects the output target (e.g., if `duration = 0` then `y = 'no'`). Yet, the duration is not known before a call is performed.  Also, after the end of the call y is obviously known.  Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
  - [Other attributes]
    - `campaign`: number of contacts performed during this campaign and for this client (numeric)
    - `pdays`: number of days that passed by after the client was last contacted from a previous campaign (numeric)
      - 999 means client was not previously contacted
    - `previous`: number of contacts performed before this campaign and for this client (numeric)
    - `poutcome`: outcome of the previous marketing campaign (categorical)

- Output variable (desired target):
  - `y`: has the client subscribed a term deposit? (binary)

> Our goal is to develop a model the best predicts the outcome `y`, the success of the marketing campaign.

> ## Question 6.  Remove the categorical variables with the most number of distinct values

In [13]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


In [36]:
df.day.unique()

array([19, 11, 16,  3,  5, 23, 14,  6, 17, 20, 13, 30, 29, 27,  7, 18, 12,
       21, 26, 22,  2,  4, 15,  8, 28,  9,  1, 10, 31, 25, 24], dtype=int64)

In [43]:
df.apply(pd.Series.nunique)

age            67
job            12
marital         3
education       4
default         2
balance      2353
housing         2
loan            2
contact         3
day            31
month          12
duration      875
campaign       32
pdays         292
previous       24
poutcome        4
c               2
dtype: int64

In [67]:
new_df = df.drop('day',1)
new_df

,age,job,marital,education,default,...,campaign,pdays,previous,poutcome,c
0,30,unemployed,married,primary,no,...,1,-1,0,unknown,no
1,33,services,married,secondary,no,...,1,339,4,failure,no
2,35,management,single,tertiary,no,...,1,330,1,failure,no
3,30,management,married,tertiary,no,...,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,...,1,-1,0,unknown,no
5,35,management,single,tertiary,no,...,2,176,3,failure,no
6,36,self-employed,married,tertiary,no,...,1,330,2,other,no
7,39,technician,married,secondary,no,...,2,-1,0,unknown,no
8,41,entrepreneur,married,tertiary,no,...,2,-1,0,unknown,no
9,43,services,married,primary,no,...,1,147,2,failure,no


Answer: TODO

> ## Question 7.  Recode all `yes`/`no` categorical variables with `0` as the most frequent value (then also append `"_no"` to the variable name), and `1` for the other (then leave the name unchanged)

In [49]:
new_df.default.value_counts()

no     4445
yes      76
Name: default, dtype: int64

In [50]:
new_df.c.value_counts()

no     4000
yes     521
Name: c, dtype: int64

In [77]:
for column in df.select_dtypes(include='object').columns()

SyntaxError: invalid syntax (<ipython-input-77-e3f553a94b67>, line 1)

> ## Question 8.  Create binary/dummy variables for the other categorical variables

In [ ]:
# TODO

> ## Question 9.  What should be your baseline for these binary variables (namely, which binary variables should you not include in your model)?

In [ ]:
# TODO

Answer: TODO

> ## Question 10.  What input variable in the dataset seems to predict the outcome quite well.  Why?

In [ ]:
# TODO

Answer: TODO

> ## Question 11.  Split the dataset into a training set (60%) and a testing set (the rest)

In [ ]:
# TODO

> ## Question 12.  Run a logistic regression with `age`, `marital` (the dummies), `default`, `balance`, `housing`, `loan`, `campaign`, `pdays`, `previous`?

In [ ]:
# TODO

> ## Question 13.  What is your training error?  What is your generalization error?  Does it make sense?

In [ ]:
# TODO

Answer: TODO

> ## Question 14.  Interpret your coefficients. (At least `marital_single`, `campaign`, and `default`).  Does your interpretation  make sense?

In [ ]:
# TODO

Answer: TODO

> ## Question 15.  What is your prediction for a 30 years old single female, a homeowner with a \$1,000 balance in the bank, without a loan, who has never been contacted before, and who has never defaulted

In [ ]:
# TODO

Answer: TODO